In [ ]:
import pandas as pd
import numpy as np
import os
import torch
import shutil
from tqdm import tqdm

from collections import Counter
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from matplotlib import pyplot
import seaborn as sns
from sklearn.metrics import accuracy_score
from statistics import mean 

In [ ]:
!pip install transformers
!pip install datasets
!pip install -U accelerate
!pip install -U transformers
!pip install evaluate

In [ ]:
from datasets import Dataset
from transformers import set_seed
SEED=42

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

In [ ]:
real_data = 'data/real_reviews/'
gpt_data = 'data/generated_reviews/'

## Preprocess data

In [ ]:
def preprocess_data(total):
  total['Downside_Review'] = total['Downside_Review'].str.replace('[^\w\s]', '', regex=True) # remove punctuation
  total['Upside_Review'] = total['Upside_Review'].str.replace('[^\w\s]', '', regex=True) # remove punctuation
  total['text'] = total['Upside_Review'].fillna('').str.strip() + ". " + total['Downside_Review'].fillna('').str.strip()  
  total['text'] = total['text'].str.lower()
  total['text'] = total['text'].str.rstrip()
  total['text'] = total['text'].str.lstrip()
  total['text'] = total['text'].str.replace("\n", " ")
  total = total.replace(np.nan, '')
  return total

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

def form_dataset(total):
  ds = Dataset.from_dict(total[['text', 'Review_Language', 'City Name', 'Sentiment', 'source']].rename(columns={'source': 'label'}).to_dict(orient='series'))
  ds = ds.map(tokenize_function, batched=True)
  ds = ds.class_encode_column('label')
  return ds

In [ ]:
def train_test_split(ds, test_size):
  ds_train_test_split = ds.train_test_split(test_size=test_size, seed=seed, stratify_by_column='label')
  train_dataset = ds_train_test_split["train"].shuffle(seed=seed)
  eval_dataset = ds_train_test_split["test"].shuffle(seed=seed)
  # Converting to a DataFrame
  train_df = pd.DataFrame(train_dataset)
  eval_df = pd.DataFrame(eval_dataset)
  pd.Series(train_dataset['label']).value_counts()
  pd.Series(eval_dataset['label']).value_counts()
  return train_dataset, eval_dataset

def train_val_test_split(ds, test_size, seed): #test_size: 0.9 -- 0.1; val_size always 0.1 from train_val
  ds_trainval_test_split = ds.train_test_split(test_size=test_size, seed=seed, stratify_by_column='label')
  ds_train_val_split = ds_trainval_test_split["train"].train_test_split(test_size=0.1, seed=seed, stratify_by_column='label')

  train_dataset = ds_train_val_split["train"].shuffle(seed=seed)
  eval_dataset = ds_train_val_split["test"].shuffle(seed=seed)
  test_dataset = ds_trainval_test_split["test"].shuffle(seed=seed)
  # Converting to a DataFrame
  train_df = pd.DataFrame(train_dataset)
  eval_df = pd.DataFrame(eval_dataset)
  test_df = pd.DataFrame(test_dataset)
  pd.Series(train_dataset['label']).value_counts()
  pd.Series(eval_dataset['label']).value_counts()
  pd.Series(test_dataset['label']).value_counts()
  return train_dataset, eval_dataset, test_dataset, test_df

In [ ]:
# Train and test for all languages
total = read_data()
total = preprocess_data(total)
ds = form_dataset(total)
columns = ['Review_Language', 'Prompt_Language', 'City Name', 'Sentiment', 'text', 'label']

test_size = 0.99 # 0.2  # change test_size from 0.99 (few shot) to 0.2 (default)
train_dataset, eval_dataset, test_dataset, test_df = train_val_test_split(ds, test_size=test_size, seed=SEED)

## Roberta-XLM deception detection model

In [ ]:
def train_eval_model(train_dataset, eval_dataset, test_dataset, zero_shot):
  set_seed(SEED)

  model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2)

  training_args = TrainingArguments(
      output_dir="real_or_fake",
      learning_rate=2e-5,
      per_device_train_batch_size=8,
      per_device_eval_batch_size=8,
      num_train_epochs=5,
      weight_decay=0.01,
      evaluation_strategy="epoch",
      save_strategy="epoch",
      load_best_model_at_end=True,
      push_to_hub=False,
      fp16=True
  )

  metric = evaluate.load("accuracy")
  def compute_metrics(eval_pred):
      logits, labels = eval_pred
      predictions = np.argmax(logits, axis=-1)
      return metric.compute(predictions=predictions, references=labels)

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics,
  )
  if not zero_shot:
      trainer.train()
    
  eval_results = trainer.evaluate()
  eval_accuracy = eval_results['eval_accuracy']
  
  if test_dataset: 
      predictions = trainer.predict(test_dataset)
      preds = np.argmax(predictions.predictions, axis=-1)
      test_accuracy = metric.compute(predictions=preds, references=predictions.label_ids)['accuracy']
  else:
      preds = []
      test_accuracy = 0

  torch.cuda.empty_cache()
  shutil.rmtree('real_or_fake') # CHECKPOINTS might mess when trying to re-train

  return eval_accuracy, test_accuracy, preds

In [ ]:
eval_accuracy, test_accuracy, predictions = train_eval_model(train_dataset, eval_dataset, test_dataset, 
                                                           zero_shot=False)
predictions = pd.DataFrame({'prediction': predictions})
all_pred = pd.concat([test_df[columns], predictions], axis=1)

eval_accuracy, test_accuracy = round(eval_accuracy * 100, 2), round(test_accuracy * 100, 2)
print(f"For test_size: {test_size}, eval accuracy: {eval_accuracy}, test accuracy: {test_accuracy}")

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
f1_score = f1_score(list(predictions['prediction']), list(test_df['label']))
acc_score = accuracy_score(list(predictions['prediction']), list(test_df['label']))
f1_score, acc_score = round(f1_score * 100, 2), round(acc_score * 100, 2)
print(f1_score, acc_score)

## Naive Bayes and Random Forest

In [ ]:
# Add the scores to initial data
def form_dataset_scores():
  total = read_csv("data/all_data_scores.csv")
  ds = Dataset.from_dict(total[['text', 'Review_Language', 'Prompt_Language', 'City Name', 'Sentiment', 'source', 'Analytic', 'Readability', 'Descriptiveness']].rename(columns={'source': 'label'}).to_dict(orient='series'))
  ds = ds.map(tokenize_function, batched=True)
  ds = ds.class_encode_column('label')
  return ds

In [ ]:
ds = form_dataset_scores()
test_size = 0.99 # 0.2 
train_dataset, eval_dataset, test_dataset, test_df = train_val_test_split(ds, test_size=test_size, seed=SEED)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.impute import SimpleImputer
import copy

def run_interpretable_models(df_train, df_test):
#     count_vect = CountVectorizer(min_df=5)  #token counts
    count_vect = TfidfVectorizer(max_features = 4500) #tf-idf scores
    
    X_train_counts = count_vect.fit_transform(df_train['text']).todense()
    X_test_counts = count_vect.transform(df_test['text']).todense()
        
    ############################# Add scores  ############################################################

    X_train_scores_r = np.asarray(df_train['Readability']).reshape(-1, 1)
    X_test_scores_r = np.asarray(df_test['Readability']).reshape(-1, 1) 

    X_train_scores_d = np.asarray(df_train['Descriptiveness']).reshape(-1, 1)
    X_test_scores_d = np.asarray(df_test['Descriptiveness']).reshape(-1, 1)
    
    X_train_scores_a = np.asarray(df_train['Analytic']).reshape(-1, 1)
    X_test_scores_a = np.asarray(df_test['Analytic']).reshape(-1, 1)
    
    X_train_scores_all = np.concatenate((X_train_scores_a, X_train_scores_d, X_train_scores_r, X_train_scores_lang), axis=1)
    X_test_scores_all = np.concatenate((X_test_scores_a, X_test_scores_d, X_test_scores_r, X_test_scores_lang), axis=1)

    X_train_all = np.concatenate((X_train_counts, X_train_scores_all), axis=1)
    X_test_all = np.concatenate((X_test_counts, X_test_scores_all), axis=1)
    
#####################################################################################################

    X_train = copy.deepcopy(X_train_scores_d) #X_train_scores_r #X_train_counts # X_train_all
    X_test = copy.deepcopy(X_test_scores_d) # X_test_scores_r #X_test_counts # X_test_all
    
    imp = SimpleImputer(missing_values=np.nan, strategy='mean') # replace missing values with mean
    imp = imp.fit(X_train)
    X_train = imp.transform(X_train)
    X_test = imp.transform(X_test)
    
#     model = RandomForestClassifier() # Choose your model
    model = GaussianNB()
    clf = model.fit(X_train, df_train['label'])
    predicted = clf.predict(X_test)

    # Accuracy
    test_accuracy = (pd.Series(predicted) == pd.Series(df_test['label'])).mean()
    test_accuracy = round(test_accuracy * 100, 2)
    f1 = f1_score(list(predicted), list(df_test['label']))
    f1 = round(f1 * 100, 2)
    print(f"Accuracy: {test_accuracy}, F1: {f1}")

    
    ######################### get salient words w. Naive Bayes ################## 
    
    prob_pos = pd.Series(df_train['label']).value_counts(normalize=True)[0] # 0 and 1 would select the corresponding labels, i.e.0 real and 1 fake
    prob_neg = pd.Series(df_train['label']).value_counts(normalize=True)[1]

    df_nbf = pd.DataFrame()
    df_nbf.index = count_vect.get_feature_names()
    
    # Convert log probabilities to probabilities. 
    df_nbf['pos'] = np.e**(clf.feature_log_prob_[0, :])
    df_nbf['neg'] = np.e**(clf.feature_log_prob_[1, :])

    df_nbf['odds_positive'] = (df_nbf['pos']/df_nbf['neg'])*(prob_pos /prob_neg)
    df_nbf['odds_negative'] = (df_nbf['neg']/df_nbf['pos'])*(prob_neg/prob_pos )

    # Here are the topX most important words for real
    odds_pos_top10 = df_nbf.sort_values('odds_positive', ascending=False)['odds_positive'][:10]

    # Here are the topX most important words for fake
    odds_neg_top10 = df_nbf.sort_values('odds_negative', ascending=False)['odds_negative'][:10]
    return odds_pos_top5, odds_neg_top5
   

In [ ]:
odds_pos_top5, odds_neg_top5 = run_interpretable_models(train_dataset, test_dataset)

print(odds_pos_top5) #real
print("-----------------") 
print(odds_neg_top5) #fake